In [44]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [2]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [3]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        df = df_res[[sp, "truth"]].dropna()
        df[sp]
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [75]:
alg_list = ["fraudar", "rsd", "rev2", "fbox", "sg"][4:]
data_list = ["alpha", "otc", "amazon", "epinions"][:2]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [84]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="ilp_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}

In [85]:
# attack_res[("epinions", "fraudar", 400.0, 0.2)].to_csv("ar.csv")

In [86]:
th_dict = {
    "fraudar": 0.5,
    "rsd": 1.2,
    "rev2": 0.1,
    "fbox": 0.5,
    "sg": 0.5,
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None, th=th_dict[a]))
    for d, a, b, f in attack_res
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
alpha sg 100.0 0.0 precision  0.177778  0.190476  0.187500  0.173913
                   recall     0.210526  0.197531  0.200000  0.173913
                   f1         0.192771  0.193939  0.193548  0.173913
               0.2 precision  0.179775  0.180723  0.187500  0.161765
                   recall     0.181818  0.161290  0.172414  0.135802
...                                ...       ...       ...       ...
otc   sg 400.0 0.6 recall     0.057915  0.054688  0.056225  0.000000
                   f1         0.091185  0.088889  0.093333  0.000000
               0.8 precision  0.214286  0.237288  0.274510  0.000000
                   recall     0.048860  0.046053  0.047138  0.000000
                   f1         0.079576  0.077135  0.080460  0.000000

[120 rows x 4 columns]

In [89]:
def get_alg_data_met(df_met, alg="rev2", data="alpha", met="f1"):
    df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])
    rdf = df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[(data, alg, slice(None), met)].droplevel(0, axis=1).droplevel([0, 1, 3], axis=0)
    rdf.index = [f"baseline-{i}" if i > 0 else "no-attack" for i in rdf.index]
    return rdf

In [92]:
get_alg_data_met(df_met, alg="sg", data="otc", met="precision")

budget,100.0,200.0,300.0,400.0
no-attack,0.181521,0.181521,0.181521,0.181521
baseline-0.2,0.182297,0.181521,0.181521,0.181521
baseline-0.4,0.182297,0.181521,0.182297,0.181521
baseline-0.6,0.178670,0.181521,0.181521,0.181521
baseline-0.8,0.175793,0.175793,0.182297,0.181521
